In [ ]:
import json
import os
import requests
import datetime
import numpy as np
import pandas as pd
import glob

In [ ]:
# Set target directory of logged API json files, will create new one if it does not exist
data_dir = 'rw_api_archive'
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [ ]:
# Download a copy of RW dataset, layer, and widget endpoints. Note 'env' and 'application' params in URL string
api_list = ['dataset','layer','widget']

for endpoint in api_list:
    url = f'http://api.resourcewatch.org/v1/{endpoint}?page[size]=10000&env=production&application=rw'
    r = requests.get(url)
    if r.status_code == 200:
        data = r.json()['data']
        mb = round(float(r.headers['content-length'])/1e6, 2)
    
        print(f"Received {mb} MB",r.headers['Content-Encoding'], "file in",
              r.headers['X-Response-Time'],f"for {endpoint} endpoint")
        d8 = datetime.datetime.now().strftime("%Y-%m-%d-%H%M%S")
        with open(f'{data_dir}/{endpoint}_{d8}.json', 'w') as fp:
            json.dump(data, fp)
    else:
        print(f"Error retrieving {endpoint} API endpoint")

In [ ]:
# select filename of most recent matching json files in target directory and load as json
recent_dataset_dl = max(glob.iglob(f'{data_dir}/datas*.json'), key=os.path.getctime)
recent_layer_dl = max(glob.iglob(f'{data_dir}/layer*.json'), key=os.path.getctime)
recent_widget_dl = max(glob.iglob(f'{data_dir}/widge*.json'), key=os.path.getctime)

with open(recent_dataset_dl) as json_file:
    rw_dataset_data = json.load(json_file)
    
with open(recent_layer_dl) as json_file:
    rw_layer_data = json.load(json_file)
    
with open(recent_widget_dl) as json_file:
    rw_widget_data = json.load(json_file)

In [ ]:
# Set substring, will search for string across all json objects
subs = 'ene_028_access_clean_cooking' #<-- SET SUBSTRING HERE
layers_using = [x for x in rw_layer_data if str(x).count(subs) != 0] 
widgets_using = [x for x in rw_widget_data if str(x).count(subs) != 0] 
datasets_using = [x for x in rw_dataset_data if str(x).count(subs) != 0] 

In [ ]:
# List ids of all matching objects
litems = [datasets_using, layers_using, widgets_using]
print(subs)
for i, x in enumerate(['datasets', 'layers', 'widgets']):
    print(x)
    for j in litems[i]:
        print('https://resourcewatch.org/admin/data/{}/{}/edit'.format(x, j['id']))